# Assess snowline altitude uncertainty

In [ ]:
import os
import glob
import xarray as xr
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import sys

## Define inputs and outputs

In [ ]:
# Import utility functions
code_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/glacier-snow-cover-analysis/'
sys.path.append(os.path.join(code_path, 'scripts'))
import utils as f

# Define path to study sites
scm_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping'

# Define path for outputs
out_path = os.path.join(scm_path, 'dataset', 'analysis')

# Get names of study sites
rgi_ids = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(scm_path, 'study-sites', 'RGI*')))]
rgi_ids

## Compile SLA bounds (uncertainty) for all sites and classifications

The original SLA was calculated by sampling the $1-AAR$ percentile of the DEM. For example, if the AAR is 0.8, the SLA is calculated as the 20th percentile of elevations over the glacier area. 

$P_{SLA} = 1-AAR$

$SLA = P_{SLA}(DEM)$

To estimate upper and lower bounds for SLA, identify "misclassified" pixels above and below the SLA, and use those to adjust the SLA percentile. 

For the upper bound, calculate the area of snow-free pixels above the SLA, convert that to a percentile relative to the total area, and add that to the original SLA percentile. Sample the $P_{upper}$ of the DEM.  

$P_{upper} = \frac{A_{snow free, above SLA}}{A_{glacier}} + P_{SLA}$

$SLA_{upper} = P_{upper}(DEM)$

For the lower bound, calculate the area of snow-covered pixels below the SLA, convert that to a percentile relative to the total area, and subtract that from the original SLA percentile. Sample the $P_{lower}$ of the DEM. 

$P_{lower} = -\frac{A_{snow covered, below SLA}}{A_{glacier}} + P_{SLA}$

$SLA_{lower} = P_{lower}(DEM)$


In [ ]:
# Define output file name
sla_bounds_fn = os.path.join(out_path, 'SLA_uncertainty_analysis.nc')
if not os.path.exists(sla_bounds_fn):
    # Initialize results DataFrame
    sla_bounds_list = []

    # Iterate over sites
    for rgi_id in tqdm(rgi_ids):
        # Load snow cover stats
        scs_fn = os.path.join(scm_path, 'study-sites', rgi_id, f"{rgi_id}_classifications.zarr")
        scs = f.load_snow_cover_stats(scs_fn)
        
        # Subset to the SLA bounds
        scs = scs[['time', 'SLA', 'SLA_lower_bound', 'SLA_upper_bound']]
        scs = scs.assign_coords({'RGIId': [rgi_id]})

        # Add to list
        sla_bounds_list.append(scs)
        
    # Concatenate into one dataset
    sla_bounds = xr.concat(sla_bounds_list, dim='time')
    
    # Save results to file
    sla_bounds.to_netcdf(sla_bounds_fn)
    print('SLA bounds saved to file:', sla_bounds_fn)
    
else:
    sla_bounds = xr.open_dataset(sla_bounds_fn)

# Add column for total range and describe stats
sla_bounds['SLA_bounds_range'] = np.abs(sla_bounds['SLA_upper_bound'] - sla_bounds['SLA_lower_bound'])
plt.hist(sla_bounds['SLA_bounds_range'], bins=100)
plt.show()  

sla_bounds